In [ ]:
import sys
sys.path.append("..")
import pytreenet as ptn
from pytreenet.utils.circuit_utils import *

import os

import numpy as np
from copy import deepcopy
import time
import pickle

import matplotlib.pyplot as plt

In [ ]:
class QEC5exp:
    def __init__(self, parameters) -> None:
        self.p = parameters
        
        num_spins=5
        virtual_spin_spin_bond_dimension = self.p["ss_vbd"]
        num_bosons_per_spin = self.p["num_bosons"]
        boson_dimension = self.p["boson_dim"]
        virtual_spin_boson_bond_dimension = self.p["sb_vbd"]

        self.state = ptn.bosonic_tree_state_for_5qec(num_spins=num_spins, virtual_spin_spin_bond_dimension=virtual_spin_spin_bond_dimension,
                                                    num_bosons_per_spin=num_bosons_per_spin, boson_dimension=boson_dimension,
                                                    virtual_spin_boson_bond_dimension=virtual_spin_boson_bond_dimension, spin_name_prefix="qubit_",
                                                    indexing=1)
        self.ref_state = deepcopy(self.state)

        if self.p["measurement"]:
            mtype = [("Z"), ("N", boson_dimension)]
        else:
            mtype = [(), ()]
        self.circ = ptn.Circuit(name="Open Circuit",
                                state=self.state, 
                                qubit_ids="qubit", 
                                print_actions=True, 
                                measurement_types=mtype)
        self.circ.progress_bar = True
        self.circ.bond_dim_tol = self.p["bond_dim_tol"]
        self.circ.print_actions = False
        self.circ.time_steps = self.p["time_steps"]

        self.hamiltonian = ptn.bosonic_tree_operator(num_spins=num_spins, num_bosons_per_spin=num_bosons_per_spin,
                                                        boson_dimension=boson_dimension, J=self.p["J"], g=self.p["g"], momega=self.p["m"],
                                                        add_extra_qubit_dim=True, spin_name_prefix="qubit_",
                                                        indexing=1)

        bosonic_index_appendix = (0,)*num_bosons_per_spin
        ts_dict = dict()
        ts_dict["qubit_1"] = ((2,)+bosonic_index_appendix,)
        ts_dict["qubit_2"] = ((2,2,)+bosonic_index_appendix,)
        ts_dict["qubit_3"] = ((2,2,)+bosonic_index_appendix,)
        ts_dict["qubit_4"] = ((2,2,)+bosonic_index_appendix,)
        ts_dict["qubit_5"] = ((2,)+bosonic_index_appendix,)
        self.gate_idx_dict = ts_dict

    def run(self):
        for _ in range(self.p["cycles"]):
            if self.p["encode"]:
                # ENCODE
                if self.p["error_during_encoding"]:
                    self.circ.provide_hamiltonian(self.hamiltonian, self.gate_idx_dict, operation_type="add")
                    ptn.qec5.encode(self.circ, self.p["encoding_time"])
                    self.circ.remove_hamiltonian()
                else:
                    ptn.qec5.encode(self.circ, self.p["encoding_time"])

            # BOSONIC ERROR
            if self.p["error_time"] > 0:
                print("Erroring")
                self.circ.provide_hamiltonian(self.hamiltonian, self.gate_idx_dict, operation_type="add")
                self.circ.gate("qubit_1", "I", operation_time=self.p["error_time"])
                self.circ.remove_hamiltonian()
            
            # RANDOM ERROR
            if self.p["error_chance"] > 0:
                r = np.random.rand()
                if r < self.p["error_chance"]:
                    qubit = np.random.choice(["qubit_"+str(i) for i in range(1, 6)])
                    error = np.random.choice(["X", "Z"])
                    self.circ.gate("qubit_1", "X", operation_time=1)

            if self.p["encode"]:
                # DECODE
                if self.p["error_during_encoding"]:
                    self.circ.provide_hamiltonian(self.hamiltonian, self.gate_idx_dict, operation_type="add")
                    ptn.qec5.decode(self.circ, self.p["encoding_time"])
                    self.circ.remove_hamiltonian()
                else:
                    ptn.qec5.decode(self.circ, self.p["encoding_time"])

                # RECOVER
                self.circ.remove_hamiltonian()
                print("Repairing")
                ptn.qec5.repair(self.circ)

        try:
            print(self.circ.fidelity(self.ref_state, mode="trace_ratio", qubit="qubit_3"))
        except:
            print("NA")

In [ ]:
params = [(0.01, 0.02, 1.5, -0-7)]

for (bdt, ts, J, g) in params:
        params = {"num_bosons": 1, 
                "ss_vbd": (6, 8),
                "sb_vbd": 4, 
                "boson_dim": 3,
                "time_steps": ts,
                "J": J,
                "g": g,
                "m": 0.1,
                "cycles": 1,
                "error_time": 1,
                "encode": True,
                "error_during_encoding": False,
                "error_chance": 0,
                "measurement": True,
                "encoding_time": .1,
                "bond_dim_tol": bdt}

        exp = QEC5exp(params)
        exp.run()
        
        filename = f"some_filename_in_results_dir.pkl"

        if filename in os.listdir("results/"):
            openmode = "ab"
        else:
            openmode = "wb"
        with open("results/"+filename, openmode) as file:
            pickle.dump(exp, file)